# 📅 Chapter 04: Working with Dates and Timestamps 

## ⏱️ Date/Time Types and Formats

### Date & Timestamp Formats

* **Date**: `YYYY-MM-DD`

  * Example: `2018-12-30`
* **Timestamp**: `YYYY-MM-DD HH:MM:SS`

  * Example: `2018-12-30 13:10:04.3`

### Intervals

* Express durations:
  Examples:
  `6 days 01:48:08`, `00:51:03`, `406 days 00:31:56`

### Common Date/Time Representations

* Examples:

  * `01/10/18 13:00`
  * `January 10th, 2018 1pm`
  * `10 Jan 2018 13:00`
* **ISO 8601** standard: `YYYY-MM-DD HH:MM:SS`
  Example: `2018-01-05 09:35:15`

### Timezones & UTC

* Timezone format: `YYYY-MM-DD HH:MM:SS+HH`
  Example: `2004-10-19 10:23:54+02`

## ⏱️ Comparing Dates and Timestamps

```sql
SELECT '2018-01-01' > '2017-12-31';
```

```sql
SELECT now() > '2017-12-31';
```

### Date Subtraction

```sql
SELECT now() - '2018-01-01';
-- Result: 343 days 21:26:32.710898
```

### Date Addition

```sql
SELECT '2010-01-01'::date + 1;
-- 2010-01-02

SELECT '2018-12-10'::date + '1 year 2 days 3 minutes'::interval;
-- 2019-12-12 00:03:00
```

## 📆 Extracting & Aggregating Date Parts

### Fields Available

* `century`, `decade`, `year`, `month`, `day`, `hour`, `minute`, `second`, `week`, `dow` (day of week)

### Extract Functions

```sql
SELECT date_part('month', now()), EXTRACT(MONTH FROM now());
```

<table style="text-align: left">
  <tr><th>date_part</th><th>date_part</th></tr>
  <tr><td>1</td><td>1</td></tr>
</table>

### Aggregating by Month Example

```sql
SELECT date_part('month', date) AS month, sum(amt)
FROM sales
GROUP BY month
ORDER BY month;
```

<table style="text-align: left">
  <tr><th>month</th><th>sum</th></tr>
  <tr><td>1</td><td>432</td></tr>
  <tr><td>3</td><td>1987</td></tr>
  <tr><td>4</td><td>3899</td></tr>
</table>

## 🧹 Truncating Dates

```sql
SELECT date_trunc('month', now());
-- 2018-12-01 00:00:00
```

```sql
SELECT date_trunc('month', date) AS month, sum(amt)
FROM sales
GROUP BY month
ORDER BY month;
```

<table style="text-align: left">
  <tr><th>month</th><th>sum</th></tr>
  <tr><td>2017-06-01</td><td>594</td></tr>
  <tr><td>2017-07-01</td><td>3824</td></tr>
</table>

## 🛠️ Generating Date/Time Series

### Generating Series (Every 2 Days)

```sql
SELECT generate_series('2018-01-01', '2018-01-15', '2 days'::interval);
```

<table style="text-align: left">
  <tr><th>generate_series</th></tr>
  <tr><td>2018-01-01 00:00:00</td></tr>
  <tr><td>2018-01-03 00:00:00</td></tr>
</table>

### End of Month Series

```sql
SELECT generate_series('2018-02-01', '2019-01-01', '1 month'::interval) - '1 day'::interval;
```

## 🧮 Aggregation by Series

```sql
SELECT date_trunc('hour', date) AS hour, count(*)
FROM sales
GROUP BY hour
ORDER BY hour;
```

<table style="text-align: left">
  <tr><th>hour</th><th>count</th></tr>
  <tr><td>2018-04-23 09:00:00</td><td>3</td></tr>
</table>

### Aggregation with Bins

```sql
WITH bins AS (
  SELECT generate_series(...) AS lower, ...
)
SELECT lower, upper, count(date)
FROM bins
LEFT JOIN sales ON date >= lower AND date < upper
GROUP BY lower, upper
ORDER BY lower;
```

<table style="text-align: left">
  <tr><th>lower</th><th>upper</th><th>count</th></tr>
  <tr><td>2018-04-23 09:00:00</td><td>2018-04-23 12:00:00</td><td>5</td></tr>
</table>

## ⏳ Time Between Events

```sql
SELECT date, lag(date) OVER (ORDER BY date), lead(date) OVER (ORDER BY date)
FROM sales;
```

```sql
SELECT date, date - lag(date) OVER (ORDER BY date) AS gap
FROM sales;
```

<table style="text-align: left">
  <tr><th>date</th><th>gap</th></tr>
  <tr><td>2018-04-23 09:13:14</td><td>00:05:41</td></tr>
</table>

### Average Time Between Events

```sql
SELECT avg(gap)
FROM (SELECT date - lag(date) OVER (ORDER BY date) AS gap FROM sales) AS gaps;
```

Result: `00:32:15.555556`

## 📈 Change in a Time Series

```sql
SELECT date, amount,
       lag(amount) OVER (ORDER BY date),
       amount - lag(amount) OVER (ORDER BY date) AS change
FROM sales;
```

<table style="text-align: left">
  <tr><th>date</th><th>amount</th><th>lag</th><th>change</th></tr>
  <tr><td>2018-04-23 09:13:14</td><td>12</td><td>31</td><td>-19</td></tr>
</table>

## 🔚 Wrap-Up Tips

* Use PostgreSQL documentation: [PostgreSQL Date/Time Functions](https://www.postgresql.org/docs/current/functions-datetime.html)
* Always explore distributions and be curious.
* Normalize, extract, truncate, and aggregate with finesse.
